In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score
import pickle

In [3]:
train = pd.read_csv('train_qWM28Yl.csv')
train.head()

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58592 entries, 0 to 58591
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   policy_id                         58592 non-null  object 
 1   policy_tenure                     58592 non-null  float64
 2   age_of_car                        58592 non-null  float64
 3   age_of_policyholder               58592 non-null  float64
 4   area_cluster                      58592 non-null  object 
 5   population_density                58592 non-null  int64  
 6   make                              58592 non-null  int64  
 7   segment                           58592 non-null  object 
 8   model                             58592 non-null  object 
 9   fuel_type                         58592 non-null  object 
 10  max_torque                        58592 non-null  object 
 11  max_power                         58592 non-null  object 
 12  engi

In [5]:
train.isna().sum().any()

False

In [6]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
policy_tenure,58592.0,0.611246,0.414156,0.002735,0.210250,0.573792,1.039104,1.396641
age_of_car,58592.0,0.069424,0.056721,0.000000,0.020000,0.060000,0.110000,1.000000
age_of_policyholder,58592.0,0.469420,0.122886,0.288462,0.365385,0.451923,0.548077,1.000000
population_density,58592.0,18826.858667,17660.174792,290.000000,6112.000000,8794.000000,27003.000000,73430.000000
make,58592.0,1.763722,1.136988,1.000000,1.000000,1.000000,3.000000,5.000000
airbags,58592.0,3.137066,1.832641,1.000000,2.000000,2.000000,6.000000,6.000000
displacement,58592.0,1162.355851,266.304786,796.000000,796.000000,1197.000000,1493.000000,1498.000000
cylinder,58592.0,3.626963,0.483616,3.000000,3.000000,4.000000,4.000000,4.000000
gear_box,58592.0,5.245443,0.430353,5.000000,5.000000,5.000000,5.000000,6.000000
turning_radius,58592.0,4.852893,0.228061,4.500000,4.600000,4.800000,5.000000,5.200000


In [7]:
for i in train.columns:
    print({i : train[i].unique()})

{'policy_id': array(['ID00001', 'ID00002', 'ID00003', ..., 'ID58590', 'ID58591',
       'ID58592'], dtype=object)}
{'policy_tenure': array([0.51587359, 0.67261851, 0.84111026, ..., 1.16227251, 1.2363069 ,
       0.12442893])}
{'age_of_car': array([0.05, 0.02, 0.11, 0.07, 0.16, 0.14, 0.04, 0.01, 0.12, 0.03, 0.1 ,
       0.09, 0.  , 0.08, 0.15, 0.13, 0.39, 0.17, 0.06, 0.18, 0.2 , 0.29,
       0.19, 0.38, 0.22, 0.23, 0.26, 0.21, 0.27, 0.28, 0.3 , 0.25, 0.31,
       0.33, 0.35, 0.32, 0.24, 0.37, 1.  , 0.42, 0.36, 0.45, 0.49, 0.46,
       0.34, 0.82, 0.44, 0.62, 0.81])}
{'age_of_policyholder': array([0.64423077, 0.375     , 0.38461538, 0.43269231, 0.63461538,
       0.51923077, 0.40384615, 0.42307692, 0.29807692, 0.44230769,
       0.59615385, 0.48076923, 0.30769231, 0.57692308, 0.60576923,
       0.47115385, 0.5       , 0.39423077, 0.61538462, 0.31730769,
       0.32692308, 0.49038462, 0.46153846, 0.92307692, 0.69230769,
       0.45192308, 0.41346154, 0.68269231, 0.36538462, 0.35576923,
  

In [8]:
cols = ['policy_id','is_adjustable_steering','is_tpms','cylinder','gear_box','is_brake_assist','is_power_door_locks',
       'is_central_locking','is_power_steering','is_driver_seat_height_adjustable','is_ecw','engine_type',
        'transmission_type']
train_data = train.drop('policy_id',axis=1)
train_data.shape

(58592, 43)

In [9]:
data_object = train_data.select_dtypes(exclude=['number'])
data_object.head()

,area_cluster,segment,model,fuel_type,max_torque,max_power,engine_type,is_esc,is_adjustable_steering,is_tpms,...,is_rear_window_washer,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert
0,C1,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,F8D Petrol Engine,No,No,No,...,No,No,No,No,No,Yes,No,No,No,Yes
1,C2,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,F8D Petrol Engine,No,No,No,...,No,No,No,No,No,Yes,No,No,No,Yes
2,C3,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,F8D Petrol Engine,No,No,No,...,No,No,No,No,No,Yes,No,No,No,Yes
3,C4,C1,M2,Petrol,113Nm@4400rpm,88.50bhp@6000rpm,1.2 L K12N Dualjet,Yes,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
4,C5,A,M3,Petrol,91Nm@4250rpm,67.06bhp@5500rpm,1.0 SCe,No,No,No,...,No,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes


In [10]:
encode = LabelEncoder()
train_data['area_cluster'] = encode.fit_transform(train_data['area_cluster'])
train_data['segment'] = encode.fit_transform(train_data['segment'])
train_data['model'] = encode.fit_transform(train_data['model'])
train_data['fuel_type'] = encode.fit_transform(train_data['fuel_type'])
train_data['max_torque'] = encode.fit_transform(train_data['max_torque'])
train_data['max_power'] = encode.fit_transform(train_data['max_power'])
train_data['is_esc'] = encode.fit_transform(train_data['is_esc'])
train_data['is_parking_sensors'] = encode.fit_transform(train_data['is_parking_sensors'])
train_data['is_parking_camera'] = encode.fit_transform(train_data['is_parking_camera'])
train_data['rear_brakes_type'] = encode.fit_transform(train_data['rear_brakes_type'])
train_data['steering_type'] = encode.fit_transform(train_data['steering_type'])
train_data['is_front_fog_lights'] = encode.fit_transform(train_data['is_front_fog_lights'])
train_data['is_rear_window_wiper'] = encode.fit_transform(train_data['is_rear_window_wiper'])
train_data['is_rear_window_washer'] = encode.fit_transform(train_data['is_rear_window_washer'])
train_data['is_rear_window_defogger'] = encode.fit_transform(train_data['is_rear_window_defogger'])
train_data['is_day_night_rear_view_mirror'] = encode.fit_transform(train_data['is_day_night_rear_view_mirror'])
train_data['is_speed_alert'] = encode.fit_transform(train_data['is_speed_alert'])
train_data['is_adjustable_steering'] = encode.fit_transform(train_data['is_adjustable_steering'])
train_data['is_tpms'] = encode.fit_transform(train_data['is_tpms'])
train_data['cylinder'] = encode.fit_transform(train_data['cylinder'])
train_data['gear_box'] = encode.fit_transform(train_data['gear_box'])
train_data['is_brake_assist'] = encode.fit_transform(train_data['is_brake_assist'])
train_data['is_power_door_locks'] = encode.fit_transform(train_data['is_power_door_locks'])
train_data['is_central_locking'] = encode.fit_transform(train_data['is_central_locking'])
train_data['is_power_steering'] = encode.fit_transform(train_data['is_power_steering'])
train_data['is_driver_seat_height_adjustable'] = encode.fit_transform(train_data['is_driver_seat_height_adjustable'])
train_data['is_ecw'] = encode.fit_transform(train_data['is_ecw'])
train_data['engine_type'] = encode.fit_transform(train_data['engine_type'])
train_data['transmission_type'] = encode.fit_transform(train_data['transmission_type'])

In [11]:
train_data.astype == 'object'

False

In [12]:
X = train_data.drop('is_claim',axis=1)
y = train_data['is_claim']

In [13]:
X.head()

,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,max_torque,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,0.515874,0.05,0.644231,0,4990,1,0,0,0,5,...,0,0,0,0,1,0,0,0,1,0
1,0.672619,0.02,0.375000,11,27003,1,0,0,0,5,...,0,0,0,0,1,0,0,0,1,0
2,0.841110,0.02,0.384615,15,4076,1,0,0,0,5,...,0,0,0,0,1,0,0,0,1,0
3,0.900277,0.11,0.432692,16,21622,1,3,3,2,0,...,1,1,1,1,1,1,1,1,1,2
4,0.596403,0.11,0.634615,17,34738,2,0,4,2,8,...,0,0,1,1,1,0,1,1,1,2


In [14]:
scaler = StandardScaler()
train_data_scale = pd.DataFrame(scaler.fit_transform(X))
train_data_scale.columns = X.columns
train_data_scale.head()

,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,max_torque,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,-0.230283,-0.342447,1.422557,-1.915921,-0.783513,-0.671712,-1.237709,-1.457229,-1.201594,0.701364,...,-0.734014,-1.103691,-1.620622,-1.620622,0.145151,-1.187895,-0.783393,-1.620622,0.078956,-1.266548
1,0.148188,-0.871359,-0.768362,-0.299191,0.462975,-0.671712,-1.237709,-1.457229,-1.201594,0.701364,...,-0.734014,-1.103691,-1.620622,-1.620622,0.145151,-1.187895,-0.783393,-1.620622,0.078956,-1.266548
2,0.555022,-0.871359,-0.690115,0.288711,-0.835268,-0.671712,-1.237709,-1.457229,-1.201594,0.701364,...,-0.734014,-1.103691,-1.620622,-1.620622,0.145151,-1.187895,-0.783393,-1.620622,0.078956,-1.266548
3,0.697883,0.715378,-0.298879,0.435686,0.158275,-0.671712,0.677613,-0.518945,1.193337,-1.347656,...,1.362372,0.906051,0.617047,0.617047,0.145151,0.841825,1.276499,0.617047,0.078956,0.172752
4,-0.035840,0.715378,1.344310,0.582662,0.900969,0.207812,-1.237709,-0.206184,1.193337,1.930776,...,-0.734014,-1.103691,0.617047,0.617047,0.145151,-1.187895,1.276499,0.617047,0.078956,0.172752


In [15]:
test = pd.read_csv('test_zo1G9sv.csv')
test.head()

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,ID58593,0.341732,0.00,0.586538,C3,4076,1,A,M1,CNG,...,No,No,No,No,Yes,No,No,No,Yes,0
1,ID58594,0.307241,0.13,0.442308,C8,8794,1,B2,M6,Petrol,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2
2,ID58595,0.327924,0.12,0.451923,C8,8794,2,A,M3,Petrol,...,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2
3,ID58596,0.782654,0.01,0.461538,C5,34738,1,A,M1,CNG,...,No,No,No,No,Yes,No,No,No,Yes,0
4,ID58597,1.233404,0.02,0.634615,C5,34738,1,A,M1,CNG,...,No,No,No,No,Yes,No,No,No,Yes,0


In [16]:
print('Test Dataset Shape :',test.shape)

Test Dataset Shape : (39063, 43)


In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39063 entries, 0 to 39062
Data columns (total 43 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   policy_id                         39063 non-null  object 
 1   policy_tenure                     39063 non-null  float64
 2   age_of_car                        39063 non-null  float64
 3   age_of_policyholder               39063 non-null  float64
 4   area_cluster                      39063 non-null  object 
 5   population_density                39063 non-null  int64  
 6   make                              39063 non-null  int64  
 7   segment                           39063 non-null  object 
 8   model                             39063 non-null  object 
 9   fuel_type                         39063 non-null  object 
 10  max_torque                        39063 non-null  object 
 11  max_power                         39063 non-null  object 
 12  engi

In [18]:
test.isna().sum().any()

False

In [19]:
test.describe().T

,count,mean,std,min,25%,50%,75%,max
policy_tenure,39063.0,0.611304,0.415005,0.002745,0.208958,0.574078,1.040936,1.391398
age_of_car,39063.0,0.069635,0.056854,0.000000,0.020000,0.060000,0.110000,1.000000
age_of_policyholder,39063.0,0.469087,0.122072,0.288462,0.375000,0.451923,0.548077,0.990385
population_density,39063.0,18816.232496,17651.942351,290.000000,6112.000000,8794.000000,27003.000000,73430.000000
make,39063.0,1.763382,1.134112,1.000000,1.000000,1.000000,3.000000,5.000000
airbags,39063.0,3.139672,1.831043,1.000000,2.000000,2.000000,6.000000,6.000000
displacement,39063.0,1160.605842,266.942754,796.000000,796.000000,1197.000000,1493.000000,1498.000000
cylinder,39063.0,3.620613,0.485241,3.000000,3.000000,4.000000,4.000000,4.000000
gear_box,39063.0,5.245782,0.430556,5.000000,5.000000,5.000000,5.000000,6.000000
turning_radius,39063.0,4.853222,0.228060,4.500000,4.600000,4.800000,5.000000,5.200000


In [20]:
for i in test.columns:
    print({i : test[i].unique()})

{'policy_id': array(['ID58593', 'ID58594', 'ID58595', ..., 'ID97653', 'ID97654',
       'ID97655'], dtype=object)}
{'policy_tenure': array([0.34173168, 0.30724144, 0.32792377, ..., 0.52028737, 1.17239103,
       0.09403029])}
{'age_of_car': array([0.  , 0.13, 0.12, 0.01, 0.02, 0.15, 0.09, 0.06, 0.1 , 0.11, 0.05,
       0.16, 0.17, 0.04, 0.24, 0.07, 0.2 , 0.03, 0.08, 0.14, 0.18, 0.21,
       0.19, 0.27, 0.31, 0.23, 0.25, 0.22, 0.34, 0.35, 0.29, 0.26, 0.33,
       0.3 , 0.36, 0.28, 0.32, 0.42, 1.  , 0.54, 0.44, 0.51, 0.37, 0.46,
       0.41, 0.43, 0.38, 0.5 , 0.48, 0.99, 0.76])}
{'age_of_policyholder': array([0.58653846, 0.44230769, 0.45192308, 0.46153846, 0.63461538,
       0.35576923, 0.47115385, 0.51923077, 0.43269231, 0.33653846,
       0.875     , 0.36538462, 0.48076923, 0.50961538, 0.57692308,
       0.34615385, 0.28846154, 0.70192308, 0.31730769, 0.375     ,
       0.56730769, 0.53846154, 0.55769231, 0.64423077, 0.5       ,
       0.41346154, 0.65384615, 0.49038462, 0.39423077, 0.

In [21]:
test_data = test.drop('policy_id',axis=1)
test_data.shape

(39063, 42)

In [22]:
test_data['is_adjustable_steering'] = encode.fit_transform(test_data['is_adjustable_steering'])
test_data['is_tpms'] = encode.fit_transform(test_data['is_tpms'])
test_data['cylinder'] = encode.fit_transform(test_data['cylinder'])
test_data['gear_box'] = encode.fit_transform(test_data['gear_box'])
test_data['is_brake_assist'] = encode.fit_transform(test_data['is_brake_assist'])
test_data['is_power_door_locks'] = encode.fit_transform(test_data['is_power_door_locks'])
test_data['is_central_locking'] = encode.fit_transform(test_data['is_central_locking'])
test_data['is_power_steering'] = encode.fit_transform(test_data['is_power_steering'])
test_data['is_driver_seat_height_adjustable'] = encode.fit_transform(test_data['is_driver_seat_height_adjustable'])
test_data['is_ecw'] = encode.fit_transform(test_data['is_ecw'])
test_data['engine_type'] = encode.fit_transform(test_data['engine_type'])
test_data['transmission_type'] = encode.fit_transform(test_data['transmission_type'])
test_data['area_cluster'] = encode.fit_transform(test_data['area_cluster'])
test_data['segment'] = encode.fit_transform(test_data['segment'])
test_data['model'] = encode.fit_transform(test_data['model'])
test_data['fuel_type'] = encode.fit_transform(test_data['fuel_type'])
test_data['max_torque'] = encode.fit_transform(test_data['max_torque'])
test_data['max_power'] = encode.fit_transform(test_data['max_power'])
test_data['is_esc'] = encode.fit_transform(test_data['is_esc'])
test_data['is_parking_sensors'] = encode.fit_transform(test_data['is_parking_sensors'])
test_data['is_parking_camera'] = encode.fit_transform(test_data['is_parking_camera'])
test_data['rear_brakes_type'] = encode.fit_transform(test_data['rear_brakes_type'])
test_data['steering_type'] = encode.fit_transform(test_data['steering_type'])
test_data['is_front_fog_lights'] = encode.fit_transform(test_data['is_front_fog_lights'])
test_data['is_rear_window_wiper'] = encode.fit_transform(test_data['is_rear_window_wiper'])
test_data['is_rear_window_washer'] = encode.fit_transform(test_data['is_rear_window_washer'])
test_data['is_rear_window_defogger'] = encode.fit_transform(test_data['is_rear_window_defogger'])
test_data['is_day_night_rear_view_mirror'] = encode.fit_transform(test_data['is_day_night_rear_view_mirror'])
test_data['is_speed_alert'] = encode.fit_transform(test_data['is_speed_alert'])

In [23]:
scaler = StandardScaler()
test_data_scale = pd.DataFrame(scaler.fit_transform(test_data))
test_data_scale.columns = test_data.columns
test_data_scale.head()

,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,max_torque,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,-0.649573,-1.224820,0.962158,0.284522,-0.835059,-0.673119,-1.227959,-1.454732,-1.195226,0.691191,...,-0.732931,-1.095497,-1.621328,-1.621328,0.137516,-1.177951,-0.778934,-1.621328,0.077963,-1.268675
1,-0.732682,1.061771,-0.219379,1.019971,-0.567777,-0.673119,0.049529,0.731191,1.199272,-1.359358,...,-0.732931,0.912828,0.616778,0.616778,0.137516,0.848932,1.283807,0.616778,0.077963,0.173628
2,-0.682845,0.885879,-0.140610,1.019971,-0.567777,0.208640,-1.227959,-0.205633,1.199272,1.921519,...,-0.732931,-1.095497,0.616778,0.616778,0.137516,-1.177951,1.283807,0.616778,0.077963,0.173628
3,0.412892,-1.048929,-0.061841,0.578701,0.901995,-0.673119,-1.227959,-1.454732,-1.195226,0.691191,...,-0.732931,-1.095497,-1.621328,-1.621328,0.137516,-1.177951,-0.778934,-1.621328,0.077963,-1.268675
4,1.499038,-0.873037,1.356003,0.578701,0.901995,-0.673119,-1.227959,-1.454732,-1.195226,0.691191,...,-0.732931,-1.095497,-1.621328,-1.621328,0.137516,-1.177951,-0.778934,-1.621328,0.077963,-1.268675


In [24]:
xtrain,xtest,ytrain,ytest = train_test_split(train_data_scale,y,test_size=0.2,random_state=41)

In [25]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(46873, 42)
(11719, 42)
(46873,)
(11719,)


# Logistic Regression

In [26]:
%%time
lr_model = LogisticRegression(class_weight='balanced')
lr_model.fit(xtrain,ytrain)

Wall time: 2.82 s


LogisticRegression(class_weight='balanced')

In [27]:
pickle.dump(lr_model, open('lr_model.pkl', 'wb'))   #LogisticRegression
# pickled_model_lr = pickle.load(open('lr_model.pkl', 'rb'))

In [28]:
lr_model_pred = lr_model.predict(xtest)
print ("\nAccuracy on validation set: {:.4f}".format(accuracy_score(ytest, lr_model_pred)*100))
print("\nClassification report : \n", classification_report(ytest, lr_model_pred))
print("\nConfusion Matrix : \n", confusion_matrix(ytest, lr_model_pred))
lr_f1_score_test = f1_score(ytest, lr_model_pred, average='weighted')*100
print("\nF1 Score : ",lr_f1_score_test)


Accuracy on validation set: 55.1583

Classification report : 
               precision    recall  f1-score   support

           0       0.95      0.55      0.70     10972
           1       0.08      0.58      0.14       747

    accuracy                           0.55     11719
   macro avg       0.52      0.56      0.42     11719
weighted avg       0.89      0.55      0.66     11719


Confusion Matrix : 
 [[6032 4940]
 [ 315  432]]

F1 Score :  66.11748521513398


# Random Forest Classifier

In [29]:
%%time
rfc_model = RandomForestClassifier(n_estimators=1000,random_state=11)
rfc_model.fit(xtrain,ytrain)

Wall time: 2min 1s


RandomForestClassifier(n_estimators=1000, random_state=11)

In [30]:
pickle.dump(rfc_model, open('rfc_model.pkl', 'wb'))   #RandomForest

In [31]:
rfc_model_pred = rfc_model.predict(xtest)
print ("\nAccuracy on validation set: {:.4f}".format(accuracy_score(ytest, rfc_model_pred)*100))
print("\nClassification report : \n", classification_report(ytest, rfc_model_pred))
print("\nConfusion Matrix : \n", confusion_matrix(ytest, rfc_model_pred))
rfc_f1_score_test = f1_score(ytest, rfc_model_pred, average='weighted')*100
print("\nF1 Score : ",rfc_f1_score_test)


Accuracy on validation set: 93.2503

Classification report : 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97     10972
           1       0.08      0.01      0.01       747

    accuracy                           0.93     11719
   macro avg       0.51      0.50      0.49     11719
weighted avg       0.88      0.93      0.90     11719


Confusion Matrix : 
 [[10924    48]
 [  743     4]]

F1 Score :  90.41830273025026


# XGBClassifier

In [32]:
%%time
xgb_model = XGBClassifier(n_estimators=1000,max_depth=6)
xgb_model.fit(xtrain,ytrain)

Wall time: 1min 42s


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=1000,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [33]:
pickle.dump(xgb_model, open('xgb_model.pkl', 'wb'))   #XGBClassifier

In [34]:
xgb_model_pred = xgb_model.predict(xtest)
print ("\nAccuracy on validation set: {:.4f}".format(accuracy_score(ytest, xgb_model_pred)*100))
print("\nClassification report : \n", classification_report(ytest, xgb_model_pred))
print("\nConfusion Matrix : \n", confusion_matrix(ytest, xgb_model_pred))
xgb_f1_score_test = f1_score(ytest, xgb_model_pred, average='weighted')*100
print("\nF1 Score : ",xgb_f1_score_test)


Accuracy on validation set: 92.9857

Classification report : 
               precision    recall  f1-score   support

           0       0.94      0.99      0.96     10972
           1       0.08      0.01      0.02       747

    accuracy                           0.93     11719
   macro avg       0.51      0.50      0.49     11719
weighted avg       0.88      0.93      0.90     11719


Confusion Matrix : 
 [[10890    82]
 [  740     7]]

F1 Score :  90.32745792042346


# Support Vector Machine

In [35]:
%%time 
svm_model = SVC(kernel='linear',class_weight='balanced',random_state=41)
svm_model.fit(xtrain,ytrain)

Wall time: 8min 20s


SVC(class_weight='balanced', kernel='linear', random_state=41)

In [36]:
pickle.dump(svm_model, open('svm_model.pkl', 'wb'))   #SupportVectorMachine

In [37]:
svm_model_pred = svm_model.predict(xtest)
print ("\nAccuracy on validation set: {:.4f}".format(accuracy_score(ytest, svm_model_pred)*100))
print("\nClassification report : \n", classification_report(ytest, svm_model_pred))
print("\nConfusion Matrix : \n", confusion_matrix(ytest, svm_model_pred))
svm_f1_score_test = f1_score(ytest, svm_model_pred, average='weighted')*100
print("\nF1 Score : ",svm_f1_score_test)


Accuracy on validation set: 53.2298

Classification report : 
               precision    recall  f1-score   support

           0       0.95      0.53      0.68     10972
           1       0.08      0.59      0.14       747

    accuracy                           0.53     11719
   macro avg       0.51      0.56      0.41     11719
weighted avg       0.89      0.53      0.64     11719


Confusion Matrix : 
 [[5794 5178]
 [ 303  444]]

F1 Score :  64.45045568925525


In [38]:
# Find out all model performance

models = pd.DataFrame({
    'model' : ['lr_model','rfc_model','xgb_model','svm_model'],
    'Score' : [lr_f1_score_test,rfc_f1_score_test,xgb_f1_score_test,svm_f1_score_test]})
round(models.sort_values(by='Score',ascending=False),2)

,model,Score
1,rfc_model,90.42
2,xgb_model,90.33
0,lr_model,66.12
3,svm_model,64.45


In [39]:
# Better performance model predict with test data

pickled_model = pickle.load(open('rfc_model.pkl', 'rb'))
final_prediction = pickled_model.predict(test_data_scale)

In [40]:
final_prediction = pd.DataFrame(final_prediction)
final_prediction.value_counts()

0    38883
1      180
dtype: int64

In [41]:
id_no = pd.DataFrame(test[test.columns[0]])
final_pred = id_no.join(final_prediction)
final_pred.to_csv('final_prediction_submission.csv',index=False)
final_pred = final_pred.rename(columns={0:'predict'})
final_pred.head()

,policy_id,predict
0,ID58593,0
1,ID58594,0
2,ID58595,0
3,ID58596,0
4,ID58597,0
